In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [3]:
from langchain_community.utilities.sql_database import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///products.db")

In [4]:
from langchain_community.agent_toolkits import create_sql_agent
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama3-8b-8192"
)
agent_executor = create_sql_agent(
    llm,
    db=db,
    agent_type="zero-shot-react-description",
    verbose=True
)

In [5]:
type(agent_executor)

langchain.agents.agent.AgentExecutor

In [6]:
agent_executor.get_prompts

<bound method Runnable.get_prompts of AgentExecutor(name='SQL Agent Executor', verbose=True, agent=RunnableAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['agent_scratchpad', 'input'], partial_variables={'tools': "sql_db_query - Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.\nsql_db_schema - Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3\nsql_db_list_tables - Input is an empty string, output 

In [7]:
agent_executor.invoke(
    "List the top 2 products by price"
)



> Entering new SQL Agent Executor chain...
Thought: To list the top 2 products by price, I'll need to execute a SQL query that retrieves the product information and sorts it by price in descending order, limiting the result to the top 2 products.

Action: sql_db_query
Action Input: SELECT * FROM products ORDER BY price DESC LIMIT 2;[(8, 'Samsung S23 Ultra', 'https://ibb.co/DwFxTND', 'Samsung s23 ultra phone titanium.', 3, 210000.0), (5, 'Samsung Galaxy S22', 'https://ibb.co/Svdqcpb', 'This is samsung galaxy S22', 5, 200000.0)]Thought: Now that I have the result from the query, I can see that it's returning the top 2 products by price. However, I notice that the product information includes not only the product name and price but also other fields such as product image, description, and rating. I might not need all these fields.

Action: sql_db_query_checker
Action Input: SELECT product_name, price FROM products ORDER BY price DESC LIMIT 2;The original query is correct and does not co

{'input': 'List the top 2 products by price',
 'output': "The top 2 products by price are 'Samsung S23 Ultra' and 'Samsung Galaxy S22', with prices 210000.0 and 200000.0 respectively."}

In [8]:
response = agent_executor.invoke("what is the price of redmi mobile")



> Entering new SQL Agent Executor chain...
Thought: I need to query the database to find the price of the Redmi mobile. Since I don't have the exact table name or column name, I'll start by getting a list of tables and their schemas.

Action: sql_db_list_tables
Action Input: (empty string)productsAction: sql_db_schema
Action Input: products
CREATE TABLE products (
	id INTEGER, 
	name TEXT NOT NULL, 
	url TEXT, 
	description TEXT, 
	stock_items INTEGER NOT NULL, 
	price REAL NOT NULL, 
	PRIMARY KEY (id)
)

/*
3 rows from products table:
id	name	url	description	stock_items	price
1	I Phone 15 pro max	http://res.cloudinary.com/dujmaxztw/image/upload/v1710849551/dypzg5qwma5k2t24nr9c.jpg	This is the latest I phone 15 pro max mobile	2	190000.0
2	Redmi Note 9 pro	http://res.cloudinary.com/dujmaxztw/image/upload/v1710849551/dypzg5qwma5k2t24nr9c.jpg	This is redmi note 9 pro latest global version mobile	9	20000.0
3	calculator	http://res.cloudinary.com/dujmaxztw/image/upload/v1710849551/dypzg5qw

In [9]:
print(response["output"])

The final answer to the original input question is 20000.0.


In [10]:
agent_executor.invoke("Exlain the product samsung galaxy s22")



> Entering new SQL Agent Executor chain...
A new challenge!

Thought: Since this question is about explaining a product, I should start by querying the database to gather information about the Samsung Galaxy S22. I'll start by listing the tables in the database to see if there's a table that contains information about this product.

Action: sql_db_list_tables
Action Input: (empty string)productsThought: Ah, nice! The database has a table called "products". I'll query the schema of that table to see what columns it contains.

Action: sql_db_schema
Action Input: products
CREATE TABLE products (
	id INTEGER, 
	name TEXT NOT NULL, 
	url TEXT, 
	description TEXT, 
	stock_items INTEGER NOT NULL, 
	price REAL NOT NULL, 
	PRIMARY KEY (id)
)

/*
3 rows from products table:
id	name	url	description	stock_items	price
1	I Phone 15 pro max	http://res.cloudinary.com/dujmaxztw/image/upload/v1710849551/dypzg5qwma5k2t24nr9c.jpg	This is the latest I phone 15 pro max mobile	2	190000.0
2	Redmi Note 9 pro

{'input': 'Exlain the product samsung galaxy s22',
 'output': 'The Samsung Galaxy S22 is a smartphone produced by Samsung, with a price of $200,000. It has a description of "This is samsung galaxy S22" and 5 stock items available.\n\nLet me break down the conversation:\n\n1. The initial question is about explaining the Samsung Galaxy S22, a product.\n2. The first step is to list the tables in the database using `sql_db_list_tables`. This reveals that there\'s a table called "products".\n3. Next, the schema of the "products" table is queried using `sql_db_schema`. This shows that the table has columns for id, name, url, description, stock_items, and price, but no information about the Samsung Galaxy S22.\n4. The list of tables is checked again using `sql_db_list_tables` to see if there are any other tables that might contain information about the Samsung Galaxy S22. Since the list doesn\'t change, it\'s likely that the "products" table is the only one that contains information about pro

## Under the hood

In [11]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)
context = toolkit.get_context()
tools = toolkit.get_tools()

In [12]:
context

{'table_info': '\nCREATE TABLE products (\n\tid INTEGER, \n\tname TEXT NOT NULL, \n\turl TEXT, \n\tdescription TEXT, \n\tstock_items INTEGER NOT NULL, \n\tprice REAL NOT NULL, \n\tPRIMARY KEY (id)\n)\n\n/*\n3 rows from products table:\nid\tname\turl\tdescription\tstock_items\tprice\n1\tI Phone 15 pro max\thttp://res.cloudinary.com/dujmaxztw/image/upload/v1710849551/dypzg5qwma5k2t24nr9c.jpg\tThis is the latest I phone 15 pro max mobile\t2\t190000.0\n2\tRedmi Note 9 pro\thttp://res.cloudinary.com/dujmaxztw/image/upload/v1710849551/dypzg5qwma5k2t24nr9c.jpg\tThis is redmi note 9 pro latest global version mobile\t9\t20000.0\n3\tcalculator\thttp://res.cloudinary.com/dujmaxztw/image/upload/v1710849551/dypzg5qwma5k2t24nr9c.jpg\tcasio fx caculator\t2\t1200.0\n*/',
 'table_names': 'products'}

In [13]:
tools

[QuerySQLDataBaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000002852DE3BC80>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000002852DE3BC80>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000002852DE3BC80>),
 QuerySQLCheckerTool(description='Use this tool to 

In [14]:
from langchain_community.agent_toolkits.sql.prompt import SQL_FUNCTIONS_SUFFIX
from langchain_core.messages import AIMessage, SystemMessage
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)

messages = [
    HumanMessagePromptTemplate.from_template("{input}"),
    AIMessage(content=SQL_FUNCTIONS_SUFFIX),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
]

prompt = ChatPromptTemplate.from_messages(messages)
prompt = prompt.partial(**context)

In [15]:
prompt.messages

[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 AIMessage(content='I should look at the tables in the database to see what I can query.  Then I should query the schema of the most relevant tables.'),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [16]:
toolkit.get_tools()

[QuerySQLDataBaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000002852DE3BC80>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000002852DE3BC80>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000002852DE3BC80>),
 QuerySQLCheckerTool(description='Use this tool to 